In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().master('local[*]').getOrCreate()

In [3]:
graph_schema = StructType([
    StructField('to', IntegerType(), False),
    StructField('from', IntegerType(), False)
])

In [4]:
pred_schema = StructType([
    StructField('vertex', IntegerType(), False),
])

In [5]:
def shortest_path(v_from, v_to, dataset_path=None):
    
    path_length = 0
    
    edges = spark.read.csv(dataset_path, sep="\t", schema=graph_schema)
    pred = spark.createDataFrame([(v_from,)], schema=pred_schema)
    
    while not pred.where(col('vertex') == v_to).collect():
        
        path_length += 1
        
        pred = edges.join(pred, col('from') == col('vertex')) \
            .select(col('to').alias('vertex')).cache()
    
    return path_length

In [6]:
print shortest_path(12, 34, "/data/twitter/twitter_sample.txt")

8
